In [8]:
import subprocess
for i in range(2428270, 2428279):
    subprocess.run(["scancel", str(i)])

scancel: error: Kill job error on job id 2428274: Access/permission denied


In [1]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_script.sh"

with open(filename) as f:
    file = f.read()


batch_sizes = [512, 256]
lrs = [5e-6, 1e-6]
num_layerss = [6]
num_headss = [8]
projection_dims = [128]

for lr in lrs:
    for batch_size in batch_sizes:
        for num_layers in num_layerss:
            for num_heads in num_headss:
                for projection_dim in projection_dims:
                    exp_name = f"coco-{num_heads}-{num_layers}-{batch_size}-{lr}"

                    result_dir = f"results/{exp_name}"
                    if not os.path.exists(result_dir):
                        os.makedirs(result_dir)

                    file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                    file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                    file = re.sub('--num_heads .*', f"--num_heads {num_heads} " + r"\\" , file)
                    file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
                    file = re.sub('--projection_dim .*', f"--projection_dim {projection_dim} " + r"\\" , file)
                    file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)

                    file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                    file = re.sub('--output .*', f"--output {result_dir}/train.log", file)
                    file = re.sub('--error .*', f"--error {result_dir}/train.log", file)

                    # if qscore_reg and "--qscore_reg \\" not in file:
                    #     file += "\n--qscore_reg \\"
                    # elif qscore_reg == False and "--qscore_reg \\" in file:
                    #     split = file.split("\n--qscore_reg \\")
                    #     file = "".join(split)

                    print(file)

                    with open(filename, 'w') as f:
                        f.write(file)

                    subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=5125e-05
#SBATCH --output results/coco-8-6-512-5e-05/train.log
#SBATCH --error results/coco-8-6-512-5e-05/train.log
#SBATCH --time=10:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 contrastive_train.py \
--vision_tokens coco_visual_tokens \
--text_tokens coco_text_tokens \
--batch_size 512 \
--lr 5e-05 \
--num_heads 8 \
--num_layers 6 \
--projection_dim 128 \
--exp_name coco-8-6-512-5e-05 \
Submitted batch job 2429020
#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results/coco-8-6-256-5e-05/train.log
#SBATCH --error results/coco-8-6-256-5e-05/train.log
#SBATCH --time=10:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 contrastive_train.py \
--vision_tokens coco_visual_tokens \
--text_tokens coco_text_t